In [1]:
import os

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[1]

file

'crawler_2.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/1.part_number/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：248


['03323',
 '912001',
 '912002',
 '912003',
 '912004',
 '912005',
 '912006',
 '912007',
 '912008',
 '912009',
 '912010',
 '912011',
 '912012',
 '912013',
 '912014',
 '912015',
 '912017',
 '912018',
 '912020',
 '912021',
 '912022',
 '912023',
 '912024',
 '912025',
 '912026',
 '912028',
 '912030',
 '912031',
 '912032',
 '912033',
 '912034',
 '912035',
 '912036',
 '912037',
 '912038',
 '912039',
 '912040',
 '912041',
 '912042',
 '912043',
 '912044',
 '912045',
 '912046',
 '912047',
 '912048',
 '912049',
 '912050',
 '912052',
 '912054',
 '912055',
 '912057',
 '912060',
 '912065',
 '912066',
 '912067',
 '912070',
 '912071',
 '912073',
 '912074',
 '912075',
 '912076',
 '912077',
 '912078',
 '912079',
 '912082',
 '912083',
 '912085',
 '912086',
 '912087',
 '912088',
 '912089',
 '912090',
 '912091',
 '912092',
 '912093',
 '912094',
 '912095',
 '912096',
 '912098',
 '912099',
 '912100',
 '912101',
 '912102',
 '912103',
 '912104',
 '912111',
 '912112',
 '912113',
 '912118',
 '912119',
 '912120',


In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'dopartsearch': '1',
        'partsearch[partnum][partsearch_007]': input_.loc[a, 'Part Number'],
        'partsearch[manufacturer][partsearch_007]': input_.loc[a, 'Manufacturer'],
        'partsearch[partgroup][partsearch_007]': input_.loc[a, 'Group'],
        'partsearch[parttype][partsearch_007]': input_.loc[a, 'Type Code'],
        'partsearch[do][partsearch_007]': '1',
        'func': 'sendparttabsearch',
        'giveback': json.dumps({}),
        'api_json_request': '1'}

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(10, 10))
        
        if resp.status_code == 200:
            json_ = resp.json()
            
            if 'redirect_to_url' not in json_ and not resp.json()['giveback'] and 'collected_javascript' in json_:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['searchnoderesults'], 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"partsearch","idepth":0,"groupname":"Cooling System","parttype":"11393","catalogname":"FVP","partnumber":"GREEN5050GAL","nodetype":"parttabsearch","expanded":true,"loaded":true}'/>
   <input autocomplete="off" id="navnodeunique[partsearch-cooling system-11393]" name="navnodeunique[partsearch-cooling system-11393]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nexpandedicon" id="navicon[__GIP__1__]">
        <a href="/en/parts/fvp,GREEN5050GAL,coolant+/+antifreeze,11393" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <img alt="Toggle" src="/Images/1pxtransparent.png"/>
        </a>
       </td>
       <td class

In [9]:
list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')

list_tbody

[<Element tbody at 0x11fa104c0>]

In [10]:
import re

list_list_note = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]') for tbody in list_tbody]
list_note = [etree.tostring(list_note[0]).decode('utf-8') if list_note else '' for list_note in list_list_note]
list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
list_note = [re.sub(r' +', ' ', note).strip() for note in list_note]

list_note

['Low Silicate Blend; 1 Gallon; Green; IAT']

In [11]:
list_dict_note = []
list_span = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]')[0] for tbody in list_tbody]

for span in list_span:
    dict_note = {}
    list_span_2 = span.xpath('./span[@class="listing-footnote-text"]')
    list_note_2 = [etree.tostring(span_2).decode('utf-8') for span_2 in list_span_2]
    list_note_2 = [re.sub(r'<.*?>', '', note_2) for note_2 in list_note_2]
    list_note_2 = [re.sub(r' +', ' ', note_2).strip() for note_2 in list_note_2]
    for i, note_2 in zip(range(len(list_note_2)), list_note_2):
        dict_note[str(i)] = note_2

    list_dict_note.append(dict_note)

list_dict_note

[{'0': 'Category: Coolant / Antifreeze'}]

In [12]:
list_json = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_json

[{'groupindex': '__GIP__3__',
  'carcode': 0,
  'parttype': '11393',
  'partkey': '13527801',
  'opts': {'22-50-0-1': {'warehouse': '95677',
    'whpartnum': 'FVP GREEN50/50-GAL',
    'optionlist': '22',
    'paramcode': '50',
    'notekey': '0',
    'multiple': '1'},
   '22-CON-0-1': {'warehouse': '95677',
    'whpartnum': 'FVP GREEN-GAL',
    'optionlist': '22',
    'paramcode': 'CON',
    'notekey': '0',
    'multiple': '1'}}}]

In [13]:
import numpy as np

array_index_type_code = np.where(np.array([json_['parttype'] for json_ in list_json]) == input_.loc[a, 'Type Code'])

array_index_type_code

(array([0]),)

In [14]:
list_part_code = [json_['partkey'] for json_ in list_json]

list_part_code

['13527801']

In [15]:
list_list_info = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]') for tbody in list_tbody]
list_info = [list_info[0].xpath('./@href')[0].strip() if list_info else '' for list_info in list_list_info]
list_info = [info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={part_code}&cc=0&pt={input_.loc[a, 'Type Code']}&Lennon=1''' for info, part_code in zip(list_info, list_part_code)]

list_info

['https://www.rockauto.com/en/moreinfo.php?pk=13527801&cc=0&pt=11393']

In [16]:
list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_json

[{'partnumber': 'GREEN5050GAL',
  'catalogname': 'FVP',
  'belongstolisting': '__GIP__2__',
  'sortgroup': 0,
  'sortgrouptext': '',
  'paramdesc': 'Type'}]

In [17]:
array_index_part_number = np.where(np.array([json_['partnumber'] for json_ in list_json]) == input_.loc[a, 'Part Number'])

array_index_part_number

(array([0]),)

In [18]:
array_index_manufacturer = np.where(np.array([json_['catalogname'] for json_ in list_json]) == input_.loc[a, 'Manufacturer'])

array_index_manufacturer

(array([0]),)

In [19]:
array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

array_index_part

array([0])

In [20]:
if len(array_index_part) == 1:
    index_part = array_index_part[0]

index_part

0

In [21]:
choose = list_json[index_part]['paramdesc'].strip()

choose

'Type'

In [22]:
list_oem = [td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')]

list_oem

[['GREENGAL, GREEN5050GAL']]

In [23]:
list_ = []
for oem in list_oem:
    if oem:
        list_ += oem[0].split(',')
list_ = [oem.strip() for oem in list_]
    
while '' in list_:
    list_.remove('')

oem = ';'.join(sorted(list(set(list_))))

oem

'GREEN5050GAL;GREENGAL'

In [24]:
list_src = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
json_ = json.loads(list_src[0].xpath('./@value')[0]) if list_src else ''

json_

{'Size': 'Thumb',
 'SlotIndex': 0,
 'Slots': [{'TitlePhrase': '<b>Image Represents Prediluted Option</b>',
   'ParamCodes': ['<b>Image Represents Prediluted Option</b>'],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/949/50 50 prediluted orig green front VP__ra_m.jpg',
    'Popup': '/info/949/50 50 prediluted orig green front VP.jpg',
    'Full': '/info/949/50 50 prediluted orig green front VP.jpg'}},
  {'TitlePhrase': '<b>Image Represents Concentrated Option</b>',
   'ParamCodes': ['<b>Image Represents Concentrated Option</b>'],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/949/full strength orig green front VP__ra_m.jpg',
    'Popup': '/info/949/full strength orig green front VP.jpg',
    'Full': '/info/949/full strength orig green front VP.jpg'}}]}

In [25]:
dict_src = {}
if json_ != '':
    list_src = json_['Slots']
    for i, src in zip(range(len(list_src)), list_src):
        dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''

dict_src

{'0': 'https://www.rockauto.com/info/949/50 50 prediluted orig green front VP.jpg',
 '1': 'https://www.rockauto.com/info/949/full strength orig green front VP.jpg'}

In [26]:
price = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
price = etree.tostring(price[0]).decode('utf-8') if price else ''
price = re.sub(r'<.*?>', '', price)
price = re.sub(r' +', ' ', price).strip()
dict_price = {'automatic': price}

dict_price

{'automatic': 'Choose Type at Left'}

In [27]:
td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')[0]
list_option = [etree.tostring(option).decode('utf-8') for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
list_option = [re.sub(r'<.*?>', '', option) for option in list_option]
list_option = [re.sub(r' +', ' ', option).strip() for option in list_option]
if list_option:
    dict_price['choose'] = {}
    for i, option in zip(range(len(list_option)), list_option):
        dict_price['choose'][str(i)] = option

dict_price

{'automatic': 'Choose Type at Left',
 'choose': {'0': 'Prediluted ($6.09) ^', '1': 'Concentrated ($8.66) ^'}}

In [28]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Group': input_.loc[a, 'Group'],
                         'Type': input_.loc[a, 'Type'],
                         'Type Code': input_.loc[a, 'Type Code'],
                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                         'Part Number': input_.loc[a, 'Part Number'],
                         'Note 1': list_note[index_part],
                         'Json_Note 2': json.dumps(list_dict_note[index_part]),
                         'Choose': choose,
                         'Json_Price': json.dumps(dict_price),
                         'OEM': oem,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Info': list_info[index_part],
                         'Json_Src': json.dumps(dict_src),
                         'Part Code': list_part_code[index_part],
                         'JOIN_MPNTCPC': f'''{input_.loc[a, 'Manufacturer']};{input_.loc[a, 'Part Number']};{input_.loc[a, 'Type Code']};{list_part_code[index_part]}'''}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Group,Type,Type Code,Manufacturer,Part Number,Note 1,Json_Note 2,Choose,Json_Price,OEM,Picture,Url,Info,Json_Src,Part Code,JOIN_MPNTCPC
0,1,Cooling System,Hood Release Cable,11393,FVP,GREEN5050GAL,Low Silicate Blend; 1 Gallon; Green; IAT,"{""0"": ""Category: Coolant / Antifreeze""}",Type,"{""automatic"": ""Choose Type at Left"", ""choose"":...",GREEN5050GAL;GREENGAL,,"https://www.rockauto.com/en/parts/dorman,03323...",https://www.rockauto.com/en/moreinfo.php?pk=13...,"{""0"": ""https://www.rockauto.com/info/949/50 50...",13527801,FVP;GREEN5050GAL;11393;13527801


In [29]:
crawler_status = 'ok'

crawler_status

'ok'

In [30]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [31]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Group,Type,Type Code,Manufacturer,Part Number,Note 1,Json_Note 2,Choose,Json_Price,OEM,Picture,Url,Info,Json_Src,Part Code,JOIN_MPNTCPC
0,1,Cooling System,Hood Release Cable,11393,FVP,GREEN5050GAL,Low Silicate Blend; 1 Gallon; Green; IAT,"{""0"": ""Category: Coolant / Antifreeze""}",Type,"{""automatic"": ""Choose Type at Left"", ""choose"":...",GREEN5050GAL;GREENGAL,,"https://www.rockauto.com/en/parts/dorman,03323...",https://www.rockauto.com/en/moreinfo.php?pk=13...,"{""0"": ""https://www.rockauto.com/info/949/50 50...",13527801,FVP;GREEN5050GAL;11393;13527801


In [32]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
